In [8]:
# библиотеки
import datetime
import pandas as pd
import numpy as np


In [197]:
# Создаем DataFrame
ddt = [datetime.datetime.strptime(date, "%d.%m.%Y") 
       for date in ['01.08.2020', '01.09.2020', '01.09.2020', '01.09.2020', '10.09.2020']]
df = pd.DataFrame({'CONTRACT_ID':[101, 101, 102, 103,103], 
              'EVENT_DT': ddt,
              'REVENUE': [200, 200, 300, 200, 150 ]})
df

,CONTRACT_ID,EVENT_DT,REVENUE
0,101,2020-08-01,200
1,101,2020-09-01,200
2,102,2020-09-01,300
3,103,2020-09-01,200
4,103,2020-09-10,150


In [201]:
# Выбираем начало месяца
df['F_day'] = df['EVENT_DT'].astype("str").str.contains("-01")
# функция условия агрегации данных
def calc_share(rev):
    denominator = np.round(rev.REVENUE.sum(), 2)
    res = {"Contract_sum_first": rev[rev['F_day'] == True]['REVENUE'].sum(),
            "Contract_sum": rev['REVENUE'].sum()}
    return pd.Series(res, index=['Contract_sum_first','Contract_sum'])
# округляем и выводим доли контрактов на первое число месяца
df = df.groupby('CONTRACT_ID').apply(calc_share)
df['Share_Contract_f_day'] = np.round(df['Contract_sum_first'] / df['Contract_sum'], 2) 
df[['Share_Contract_f_day']]

,Share_Contract_f_day
CONTRACT_ID,
101,1.00
102,1.00
103,0.57


### 2 task

In [252]:
# Создаем DataFrame 
START_DT = [datetime.datetime.strptime(date, "%d.%m.%Y") 
       for date in ['01.01.2020', '01.07.2020', '01.09.2020', '01.08.2020', '01.01.2020']]
END_DT = [datetime.datetime.strptime(date, "%d.%m.%Y") 
       for date in ['01.07.2020','01.09.2020', '31.12.2099', '31.12.2099', '31.12.2099']]
df_CONTRACTS_PRODUCTS = pd.DataFrame({'CONTRACT_ID':['101', '101', '101', '102','103'], 
              'PRODUCT_ID': [1, 3, 4, 2, 4],
              'START_DT': START_DT,
                'END_DT': END_DT})
df_CONTRACTS_PRODUCTS

,CONTRACT_ID,PRODUCT_ID,START_DT,END_DT
0,101,1,2020-01-01,2020-07-01
1,101,3,2020-07-01,2020-09-01
2,101,4,2020-09-01,2099-12-31
3,102,2,2020-08-01,2099-12-31
4,103,4,2020-01-01,2099-12-31


In [253]:
df_PRODUCT = pd.DataFrame({'PRODUCT_ID': [1, 2,3,4,5],
             'PRODUCT_NAME': ['o2 Blue S','o2 Blue M','o2 Free S','o2 Free M','o2 Free L']})
df_PRODUCT

,PRODUCT_ID,PRODUCT_NAME
0,1,o2 Blue S
1,2,o2 Blue M
2,3,o2 Free S
3,4,o2 Free M
4,5,o2 Free L


In [267]:
# объединяем обе таблицы и создаем колонку булевыми значениями под актуальные контракты
today = pd.to_datetime(datetime.date.today())
filter_date = (pd.to_datetime(df_CONTRACTS_PRODUCTS['START_DT']) < today) & \
(pd.to_datetime(df_CONTRACTS_PRODUCTS['END_DT']) > today)
df_CONTRACTS_PRODUCTS['is_works'] = filter_date
df_tmp = pd.merge(df_CONTRACTS_PRODUCTS, df_PRODUCT, on='PRODUCT_ID', how='outer').fillna(False)
df_tmp.tail()

,CONTRACT_ID,PRODUCT_ID,START_DT,END_DT,is_works,PRODUCT_NAME
1,101,3,2020-07-01 00:00:00,2020-09-01 00:00:00,False,o2 Free S
2,101,4,2020-09-01 00:00:00,2099-12-31 00:00:00,True,o2 Free M
3,103,4,2020-01-01 00:00:00,2099-12-31 00:00:00,True,o2 Free M
4,102,2,2020-08-01 00:00:00,2099-12-31 00:00:00,True,o2 Blue M
5,False,5,False,False,False,o2 Free L


In [310]:
df_tmp2= df_tmp[['CONTRACT_ID','PRODUCT_ID','is_works', 'PRODUCT_NAME']]
# функция условия агрегации данных
def is_available(table):
    res = {'CONTRACTS': table[table['is_works'] == True]['CONTRACT_ID'].count(),
          'CONTRACT_ID': table[table['is_works'] == True]['CONTRACT_ID'].apply(lambda x: x + ', ').sum(),
          'PRODUCT_NAME': table[table['is_works'] == True]['PRODUCT_NAME'].apply(lambda x: x + ', ').sum()}
    return pd.Series(res, index=['CONTRACT_ID','CONTRACTS', 'PRODUCT_NAME'])
# группируем и применяем функцию
df_res = df_tmp2.groupby('PRODUCT_ID').apply(is_available)
df_res['CONTRACT_ID'] = df_res['CONTRACT_ID'].str.rstrip(' ,')
df_res['PRODUCT_NAME'] = df_res['PRODUCT_NAME'].str.rstrip(' ,')
df_res

,CONTRACT_ID,CONTRACTS,PRODUCT_NAME
PRODUCT_ID,,,
1,NaN,0,NaN
2,102,1,o2 Blue M
3,NaN,0,NaN
4,"101, 103",2,"o2 Free M, o2 Free M"
5,NaN,0,NaN


### 3 task

In [366]:
# Создаем DataFrame
ddt = [datetime.datetime.strptime(date, "%d.%m.%Y") 
       for date in 
       ['10.01.2020', '20.01.2020', '30.12.2019', '10.02.2020', '11.02.2020', '11.02.2020', '30.12.2019', '10.01.2020']]
df = pd.DataFrame({'Manager':[1, 1, 1, 2, 2, 2, 3, 3], 
              'Product': [10, 20, 30, 10, 20, 30, 20, 30],
              'Sale': [500, 300, 500, 400, 500, 500, 500, 300],
              'Date': ddt})

df

,Manager,Product,Sale,Date
0,1,10,500,2020-01-10
1,1,20,300,2020-01-20
2,1,30,500,2019-12-30
3,2,10,400,2020-02-10
4,2,20,500,2020-02-11
5,2,30,500,2020-02-11
6,3,20,500,2019-12-30
7,3,30,300,2020-01-10


In [369]:
#Сортируем по 2 полям из условия в порядке убывания и оставляем только уникальных менеджеров (первую встречу в датасете)
df.sort_values(by=['Sale', 'Date'],ascending=False, inplace=True)
df.drop_duplicates(subset=['Manager'], keep='first').reset_index(drop=True)

,Manager,Product,Sale,Date
0,2,20,500,2020-02-11
1,1,10,500,2020-01-10
2,3,20,500,2019-12-30
